In [1]:
from pathlib import Path
import gdsfactory as gf

In [2]:
from ihp import PDK, cells
PDK.activate()

In [ ]:
GDS_PATH = Path('../gds').resolve()
GDS_PATH

PosixPath('/mnt/c/Users/dasdi/Documents/GDSFactory/diogo/cells/design_data/gds')

In [8]:
PDK.layer_stack.layers.keys()

dict_keys(['substrate', 'active', 'poly', 'metal1', 'via1', 'metal2', 'via2', 'metal3', 'via3', 'metal4', 'via4', 'metal5', 'topvia1', 'topmetal1', 'topvia2', 'topmetal2'])

In [ ]:

import gdsfactory as gf
from gdsfactory import Component
from gdsfactory.typings import LayerSpec
from typing import Optional

from ihp import tech
from ihp.cells import via_stack

def snap_to_grid(p, grid: float = 0.005):
    return round(p / grid) * grid

def cmom_extractor(
    nfingers: int = 1,
    length: float = 2.0,
    spacing: float = 0.26,
    min_width: float = 0.2,
    mom_metals: list[LayerSpec] = [],
    **kwargs
) -> float:
    
    layer_thickness = {
        'Metal1': tech.LAYER_STACK['metal1'].thickness,
        'Metal2': tech.LAYER_STACK['metal2'].thickness,
        'Metal3': tech.LAYER_STACK['metal3'].thickness,
        'Metal4': tech.LAYER_STACK['metal4'].thickness,
        'Metal5': tech.LAYER_STACK['metal5'].thickness,
    }

    total_cap: float = 0.0
    fringe_field_factor = kwargs.get('fringe_field_factor', 0.2)
    permitivity = kwargs.get('interlayer_dielectric', 3.8)
    eps0 = 8.8541878188e-3 # 

    for metal in mom_metals:
        th = layer_thickness[metal]
        min_finger_width = min_width
        total_cap += permitivity*eps0*(th / spacing)* \
            ( ( nfingers * 2 ) * (length + spacing) + min_finger_width*(nfingers*2 + 1) )
    return total_cap * (1 + fringe_field_factor)


@gf.cell
def cmom(
    nfingers: int = 1,
    length: float = 4.0,
    spacing: float = 0.26,
    botmetal: LayerSpec = "Metal1",
    topmetal: LayerSpec = "Metal3",
    layer_cap_mark: LayerSpec = "MemCapdrawing",
    layer_text: LayerSpec = "TEXTdrawing",
    model: str = 'cmom',
    **kwargs
) -> Component:
    

    metals = {
        'Metal1': 'Metal1drawing',
        'Metal2': 'Metal2drawing',
        'Metal3': 'Metal3drawing',
        'Metal4': 'Metal4drawing',
        'Metal5': 'Metal5drawing',
    }

    pdk_design_rules = {
        'Metal1':{'min_width':tech.TECH.metal1_width, 'min_spacing':tech.TECH.metal1_spacing},
        'Metal2':{'min_width':tech.TECH.metal2_width, 'min_spacing':tech.TECH.metal2_spacing},
        'Metal3':{'min_width':tech.TECH.metal3_width, 'min_spacing':tech.TECH.metal3_spacing},
        'Metal4':{'min_width':tech.TECH.metal4_width, 'min_spacing':tech.TECH.metal4_spacing},
        'Metal5':{'min_width':tech.TECH.metal5_width, 'min_spacing':tech.TECH.metal5_spacing}
    }
    
    min_width_global = min([v['min_width'] for v in pdk_design_rules.values()])
    min_spacing_global = min([v['min_spacing'] for v in pdk_design_rules.values()])
    min_length = 3 * min_width_global # to comply with minimum metal area DRC

    assert length > min_length, f"Minimum Area for all metals > {min_length * min_width_global}"
    assert spacing > min_spacing_global, f"Minimum metal spacing for all metals > {min_spacing_global}"

    ordered_metals = list(metals.keys())
    #ordered_vias = [lay for lay in ordered_layers if "via" in lay]
    assert botmetal in ordered_metals, "{botmetal} not it available layers: {_ordered_metals}"
    assert topmetal in ordered_metals, "{topmetal} not it available layers: {_ordered_metals}"
    mom_metals = ordered_metals[
        ordered_metals.index(botmetal) : ordered_metals.index(topmetal)+1
    ]
    c = gf.Component()
    total_length = 0.0
    top_pad_ref = None
    bot_pad_ref = None
    for metal_layer in mom_metals:
        min_width = min_width_global
        layer = metals[metal_layer]
        top_finger = gf.components.rectangle(
            size=(min_width, length), layer=layer)
        top_finger_array = c.add_ref(
            top_finger, columns=nfingers+1, rows=1, 
            column_pitch=2*(spacing+min_width))
        top_finger_array.ymin += spacing
        bot_finger = gf.components.rectangle(
            size=(min_width, length), layer=layer)
        bot_finger_array = c.add_ref(
            bot_finger, columns=nfingers, rows=1, 
            column_pitch=2*(spacing+min_width))
        bot_finger_array.xmin += min_width + spacing      
        total_length = (min_width + spacing) * (2*nfingers + 1) - spacing
        top_pad = gf.components.rectangle(
            size=(total_length, 3*min_width), layer=layer)
        top_pad_ref = c.add_ref(top_pad)
        top_pad_ref.ymin += length + spacing
        bot_pad_ref = c.add_ref(top_pad)
        bot_pad_ref.ymax = 0
        
        
        #   add no fill and no QRC layers to the mom device region
        nofill_layer = metal_layer.capitalize()+'nofill'
        c.add_ref(gf.components.bbox(
            c, layer=nofill_layer
        ))


    # add capacitor region marker
    c.add_ref(gf.components.bbox(
        c, layer=layer_cap_mark
    ))

    # add ports 
    pin_layer: LayerSpec = metal_layer.capitalize()+'pin'
    label_layer: LayerSpec = metal_layer.capitalize()+'label'
    
    c.add_port(
        "PLUS",
        center=(top_pad_ref.x, top_pad_ref.y), 
        width=min_width, layer=pin_layer)
    c.add_port(
        "MINUS",
        center=(bot_pad_ref.x, bot_pad_ref.y), 
        width=min_width, layer=pin_layer)
    
    c.add_label(
        text="PLUS",
        position=(top_pad_ref.x, top_pad_ref.y),
        layer=label_layer
    )
    c.add_label(
        text="MINUS",
        position=(bot_pad_ref.x, bot_pad_ref.y),
        layer=label_layer
    )
    c.add_label(
        text="PLUS",
        position=(top_pad_ref.x, top_pad_ref.y),
        layer=layer_text
    )
    c.add_label(
        text="MINUS",
        position=(bot_pad_ref.x, bot_pad_ref.y),
        layer=layer_text
    )

    c.add_label(
        text=model,
        position=(c.x, c.y + min_width),
        layer=layer_text
    )
    

    #   add a via array stack to the top and bottom pads
    mom_via_stack = via_stack(
        bottom_layer = botmetal.capitalize(),
        top_layer = topmetal.capitalize(),
        size = (total_length, 3*min_width),
        vn_columns = nfingers * 4,
        vn_rows = 1
    )

    via_stack_bot = c.add_ref(mom_via_stack)
    via_stack_bot.xmin = bot_pad_ref.xmin
    via_stack_bot.ymax = bot_pad_ref.ymax
    via_stack_top = c.add_ref(mom_via_stack)
    via_stack_top.xmin = top_pad_ref.xmin
    via_stack_top.ymin = top_pad_ref.ymin
    
    #   add place and route layers to define the device's bounding box
    prboundary_layer = 'prBoundarydrawing'
    c.add_ref(gf.components.bbox(
        c, layer=prboundary_layer
    ))
    c.info['capacitance'] = cmom_extractor(
        nfingers,
        length,
        spacing,
        min_width = min_width_global,
        mom_metals = mom_metals,
        **kwargs
        )
    c.add_label(
        text=f'C = {c.info['capacitance']} fF',
        position=(c.x, c.y - min_width),
        layer=layer_text
    )
    c.info['model'] = 'cmom'
    c.info['nfingers'] = nfingers
    c.info['length'] = length
    c.info['spacing'] = spacing
    
    #   return the component
    return c

In [ ]:
# Create a new component
c = cmom(nfingers=2)
c.info

Info(capacitance=3.8016569665500213, model='cmom', nfingers=2, length=4.0, spacing=0.26)

In [ ]:
# Doubling the number of fingers should double the estimated capacitance.
c2 = cmom(nfingers=4)
c2.info

Info(capacitance=7.573312130658273, model='cmom', nfingers=4, length=4.0, spacing=0.26)

In [ ]:
c.write(GDS_PATH / "cmom.gds")

## Extract Capacitance using a field solver to confirm the estimated value is within 10% error

Note: In reality, a process file with multiple patterns measured from the factory should be shipped with the PDK's files to use a Decision Tree regressor to build a dedicated extractor for the component.

The Decision Tree receives as inputs:
    - Number of fingers,
    - Spacing,
    - Width,
    - Layer stack
and outputs:
    - Capacitance


Each special PCell is usually shipped with its dedicated extractor using factory-control files as an index we can access and train the models with, in modern, advanced PDKs.

In [ ]:
from math import inf
import numpy as np

from gdsfactory.component import Component
from gdsfactory.technology import LayerStack

from gplugins.common.base_models.simulation import ElectrostaticResults
from gplugins.elmer import run_capacitive_simulation_elmer

In [ ]:
layer_stack:LayerStack = PDK.get_layer_stack()

material_spec = {
    "si": {"relative_permittivity": 11.45},
    "aluminium": {"relative_permittivity": inf},
    "ild": {"relative_permittivity": 3.8}, # inter layer dielectric
    "vacuum": {"relative_permittivity": 1},
}

In [ ]:
layer_stack

LayerStack(layers={'substrate': LayerLevel(name=None, layer=Substratedrawing, derived_layer=None, thickness=300.0, thickness_tolerance=None, width_tolerance=None, zmin=-300.0, zmin_tolerance=None, sidewall_angle=0.0, sidewall_angle_tolerance=None, width_to_z=0.0, z_to_bias=None, bias=None, mesh_order=3, material='si', info={'mesh_order': 99}), 'active': LayerLevel(name=None, layer=Activdrawing, derived_layer=None, thickness=0.2, thickness_tolerance=None, width_tolerance=None, zmin=0.0, zmin_tolerance=None, sidewall_angle=0.0, sidewall_angle_tolerance=None, width_to_z=0.0, z_to_bias=None, bias=None, mesh_order=3, material='si', info={'mesh_order': 1}), 'poly': LayerLevel(name=None, layer=GatPolydrawing, derived_layer=None, thickness=0.18, thickness_tolerance=None, width_tolerance=None, zmin=0.0, zmin_tolerance=None, sidewall_angle=0.0, sidewall_angle_tolerance=None, width_to_z=0.0, z_to_bias=None, bias=None, mesh_order=3, material='poly_si', info={'mesh_order': 2}), 'metal1': LayerLevel

Build geometry:

In [ ]:
simulation_box = np.array([[-200, -200], [200, 200]])
c = Component()
cap = c << cmom(nfingers=2, botmetal='Metal1', topmetal='Metal5')
c.add_ports(cap.ports)
ds = simulation_box[1,:] - simulation_box[0,:]
bbox = c << gf.components.rectangle(size=(tuple(ds)), layer = PDK.layers.SiWGdrawing)
bbox.xmin = simulation_box[0,0]
bbox.ymin = simulation_box[0,1]
substrate = gf.components.bbox(bbox, layer=PDK.layers.SiWGdrawing)
_ = c << substrate
c.flatten()
c.show()

2025-12-11 23:43:21.468 | WARNING  | kfactory.kcell:show:4022 - Could not connect to klive server


In [ ]:
def get_reasonable_mesh_parameters(c: Component) -> dict:
    return dict(
        #background_tag="vacuum",
        #background_padding=(0,) * 5 + (700,),
        #port_names=[port.name for port in c.ports],
        default_characteristic_length=200,
        resolution_specs={
            "bw": {
                "resolution": 15,
            },
            "substrate": {
                "resolution": 40,
            },
            "vacuum": {
                "resolution": 40,
            },
            **{
                f"bw__{port}": {
                    "resolution": 20,
                    "DistMax": 30,
                    "DistMin": 10,
                    "SizeMax": 14,
                    "SizeMin": 3,
                }
                for port in c.ports
            },
        },
    )

In [ ]:
mesh_params = get_reasonable_mesh_parameters(c)
mesh_params

{'default_characteristic_length': 200,
 'resolution_specs': {'bw': {'resolution': 15},
  'substrate': {'resolution': 40},
  'vacuum': {'resolution': 40},
  "bw__DPort(self.name='PLUS', self.width=0.14, trans=r0 *1 0.87,4.47, layer=Metal5pin (67/2), port_type=optical)": {'resolution': 20,
   'DistMax': 30,
   'DistMin': 10,
   'SizeMax': 14,
   'SizeMin': 3},
  "bw__DPort(self.name='MINUS', self.width=0.14, trans=r0 *1 0.87,-0.21, layer=Metal5pin (67/2), port_type=optical)": {'resolution': 20,
   'DistMax': 30,
   'DistMin': 10,
   'SizeMax': 14,
   'SizeMin': 3}}}

In [ ]:
def elmer_capacitance_simulation_basic_results(geometry: Component, layer_stack:LayerStack, material_spec:dict) -> ElectrostaticResults:
    """Run a Elmer capacitance simulation and cache the results."""
    c = geometry
    return run_capacitive_simulation_elmer(
        c,
        layer_stack=layer_stack,
        material_spec=material_spec,
        mesh_parameters=get_reasonable_mesh_parameters(c),
    )

In [ ]:
cap_results: ElectrostaticResults = elmer_capacitance_simulation_basic_results(
    c, layer_stack, material_spec
)
cap_results

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Processing ('metal1',) - instantiation
Processing ('via1',) - instantiation                                                                                                      
Processing ('metal2',) - instantiation                                                                                                    
Processing ('via2',) - instantiation                                                                                                      
Processing ('metal3',) - instantiationor internal shapes                                                                                  
Processing ('via3',) - instantiation for internal shapes                                                                                  
Processing ('metal4',) - instantiationor internal shapes                                                                                  
Processing ('via4',) - instantiation   

NotImplementedError: 237 is not a <class 'gdsfactory.technology.layer_stack.LogicalLayer'>